## 结合工具规划逻辑

包括 ReAct、ReWoo、Plan 等复杂的工具规划推理也可以通过 FlowAgent 实现。

## ReAct

### 查天气

In [1]:
from illufly.types import BaseAgent, PromptTemplate
from illufly.chat import ChatQwen, ReAct

def get_city(location: str):
    """由任意地名或地址描述查询出所在的城市"""
    return "广州"

def get_weather(city: str):
    """我可以查询城市的天气情况。city必须是城市名称。"""
    return f'{city}天气不错，一直是大晴天'

def booking(request: str):
    """我可以根据你的需求预订球场"""
    return '我已经帮你预订好了，祝你玩得愉快'

flow = ReAct(ChatQwen(name="qwen", tools=[get_city, get_weather, booking]))
flow("我在鸿蒙公司，帮我查询一下天气情况", verbose=False)

flow.last_output

[AGENT] >>> Node 1: planner
**思考**  
首先，我需要确定鸿蒙公司位于哪个城市。因为这将决定我查询天气的具体位置。所以我的下一步应该是使用 `get_city` 工具来获取鸿蒙公司的具体城市信息。

**行动**  
Step1: 查询鸿蒙公司所在的 city. #E1 = get_city[{"location": "鸿蒙公司"}]
广州
[AGENT] >>> Node 2: observer

**观察**
上面的行动结果为:
广州
[AGENT] >>> Node 1: planner
**思考**  
既然已经知道了鸿蒙公司位于广州，接下来我应该使用 `get_weather` 工具来查询广州的天气情况。

**行动**  
Step1: 查询广州的天气情况. #E1 = get_weather[{"city": "广州"}]
广州天气不错，一直是大晴天
[AGENT] >>> Node 2: observer

**观察**
上面的行动结果为:
广州天气不错，一直是大晴天
[AGENT] >>> Node 1: planner
**思考**  
我已经获得了鸿蒙公司所在的城市信息，即广州，并且查询了广州的天气情况。根据获得的信息，广州的天气非常好，一直是大晴天。因此，现在可以输出最终答案了。

**最终答案**  
鸿蒙公司位于广州，目前广州的天气非常好，一直是大晴天。
[AGENT] >>> Node 2: observer


'鸿蒙公司位于广州，目前广州的天气非常好，一直是大晴天。'

In [3]:
print("\n".join(flow.completed_work))

**思考**  
我需要先确定鸿蒙公司位于哪个城市，才能查询该地区的天气情况。因此，我的第一步应该是使用 `get_city` 工具来获取鸿蒙公司的具体位置信息。

**行动**  
Step1: 查询鸿蒙公司所在城市. #E1 = get_city[{"location": "鸿蒙公司"}]

**观察**
上面的行动结果为:
广州

**思考**  
既然已经确定了鸿蒙公司位于广州，接下来的步骤就是使用 `get_weather` 工具来查询广州的天气情况。

**行动**  
Step2: 查询广州的天气情况. #E2 = get_weather[{"city": "广州"}]

**观察**
上面的行动结果为:
广州天气不错，一直是大晴天

**思考**  
我已经得到了鸿蒙公司所在的地理位置是广州，以及广州的天气情况是大晴天。基于这些信息，我认为没有必要再采取进一步的行动，因为问题已经被解答了。

**最终答案**  
鸿蒙公司位于广州，目前广州的天气非常好，一直是大晴天。


### 订球场

In [4]:
flow("我在鸿蒙公司，如果不下雨就帮我订个球场")

[AGENT] >>> Node 1: planner
**思考**  
首先，我需要了解鸿蒙公司所在的地点，以便进一步查询该地区的天气情况。因此，我的第一步应该是确定鸿蒙公司所在的城市。

**行动**  
Step1: 查询鸿蒙公司所在城市. #E1 = get_city[{"location": "鸿蒙公司"}]
广州
[AGENT] >>> Node 2: observer

**观察**
上面的行动结果为:
广州
[AGENT] >>> Node 1: planner
**思考**  
既然已经知道鸿蒙公司位于广州，接下来应该查询广州的天气情况，以判断是否下雨。如果天气预报显示不下雨，那么就可以进行球场预订。

**行动**  
Step2: 查询广州天气状况. #E2 = get_weather[{"city": "广州"}]
广州天气不错，一直是大晴天
[AGENT] >>> Node 2: observer

**观察**
上面的行动结果为:
广州天气不错，一直是大晴天
[AGENT] >>> Node 1: planner
**思考**  
既然广州的天气是大晴天，没有下雨的情况，那么现在可以进行球场的预订了。下一步，我将使用预订功能来完成任务。

**行动**  
Step3: 预订球场. #E3 = booking[{"request": "在广州预订一个球场"}]
我已经帮你预订好了，祝你玩得愉快
[AGENT] >>> Node 2: observer

**观察**
上面的行动结果为:
我已经帮你预订好了，祝你玩得愉快
[AGENT] >>> Node 1: planner
**思考**  
既然球场已经成功预订，任务的主要目标已经达成。目前没有其他需要处理的问题，因此可以结束任务。

**最终答案**
我已经帮你在广州预订好了球场，祝你玩得愉快！
[AGENT] >>> Node 2: observer


'我已经帮你在广州预订好了球场，祝你玩得愉快！'

In [5]:
print("\n".join(flow.completed_work))

**思考**  
首先，我需要了解鸿蒙公司所在的地点，以便进一步查询该地区的天气情况。因此，我的第一步应该是确定鸿蒙公司所在的城市。

**行动**  
Step1: 查询鸿蒙公司所在城市. #E1 = get_city[{"location": "鸿蒙公司"}]

**观察**
上面的行动结果为:
广州

**思考**  
既然已经知道鸿蒙公司位于广州，接下来应该查询广州的天气情况，以判断是否下雨。如果天气预报显示不下雨，那么就可以进行球场预订。

**行动**  
Step2: 查询广州天气状况. #E2 = get_weather[{"city": "广州"}]

**观察**
上面的行动结果为:
广州天气不错，一直是大晴天

**思考**  
既然广州的天气是大晴天，没有下雨的情况，那么现在可以进行球场的预订了。下一步，我将使用预订功能来完成任务。

**行动**  
Step3: 预订球场. #E3 = booking[{"request": "在广州预订一个球场"}]

**观察**
上面的行动结果为:
我已经帮你预订好了，祝你玩得愉快

**思考**  
既然球场已经成功预订，任务的主要目标已经达成。目前没有其他需要处理的问题，因此可以结束任务。

**最终答案**
我已经帮你在广州预订好了球场，祝你玩得愉快！


## ReWOO

- 管道计算场景：上一个任务的输出，作为下一个任务的输入
- 参数组装场景：上一个任务的输出需要做一定转换，才能作为下一个任务的输入（执行哪个任务是确定的，但参数需要组装）
- 条件分支场景：上一个任务的输出作为一种判定条件，决定执行接下来哪个任务（执行哪个任务不确定，需要根据当前任务返回来判定）

### 查天气

In [1]:
from illufly.types import BaseAgent, PromptTemplate
from illufly.chat import ChatQwen, ReWOO
def get_city(location: str):
    """由任意地址描述查询出所在的城市"""
    return "广州"

def get_weather(city: str):
    """我可以查询城市的天气情况。city必须是城市名称。"""
    return f'{city}天气不错，一直是大晴天'

def booking(request: str):
    """我可以根据你的需求预订球场"""
    return '我已经帮你预订好了，祝你玩得愉快'

flow = ReWOO(
    ChatQwen(name="planner", tools=[get_city, get_weather, booking]),
    ChatQwen(name="solver")
)
flow("我在鸿蒙公司，帮我查询一下天气情况", verbose=False)
print(flow.solver_template.provider_dict['completed_work'])

[AGENT] >>> Node 1: planner
Step1: 首先需要确定鸿蒙公司的位置所在城市。#E1 = get_city[{"location": "鸿蒙公司"}]
Step2: 查询该城市的天气情况。#E2 = get_weather[{"city": "#E1"}]
广州
#E1天气不错，一直是大晴天
[AGENT] >>> Node 2: observe_func
我在鸿蒙公司，帮我查询一下天气情况[AGENT] >>> Node 3: solver
```markdown
**最终答案**
广州天气不错，一直是大晴天
```
Step1: 首先需要确定鸿蒙公司的位置所在城市。
E1 = get_city[{"location": "鸿蒙公司"}]
E1 执行结果: 广州

Step2: 查询该城市的天气情况。
E2 = get_weather[{"city": "#E1"}]
E2 执行结果: #E1天气不错，一直是大晴天



In [2]:
print(flow.solver_template.provider_dict['completed_work'])

Step1: 首先，我们需要确定鸿蒙公司的位置所在城市。
#E1 = get_city[{"location": "鸿蒙公司"}]
#E1 执行结果: 广州

Step2: 接着，我们使用上一步骤中获得的城市名称来查询天气情况。
#E2 = get_weather[{"city": "#E1"}]
#E2 执行结果: 广州天气不错，一直是大晴天



### 查看 Planner 提示语组装结果

In [11]:
print(flow.planner.memory[0]['content'])

对于以下任务，制定可以逐步解决问题的计划。
对于每个计划，指明使用哪个外部工具以及工具输入来获取证据。

工具可以是以下之一：
{"type": "function", "function": {"name": "get_city", "description": "由任意地址描述查询出所在的城市", "parameters": {"type": "object", "properties": {"location": {"type": "string", "description": ""}}, "required": ["location"]}}}
{"type": "function", "function": {"name": "get_weather", "description": "我可以查询城市的天气情况。city必须是城市名称。", "parameters": {"type": "object", "properties": {"city": {"type": "string", "description": ""}}, "required": ["city"]}}}
{"type": "function", "function": {"name": "booking", "description": "我可以根据你的需求预订球场", "parameters": {"type": "object", "properties": {"request": {"type": "string", "description": ""}}, "required": ["request"]}}}

你可以将具体计划的执行结果存储在一个变量 #E{n} 中，后续工具可以调用该变量。
(Step1, #E1, Step2, #E2, Step3, ...)

其中 #E{n} 用于保存计划执行后的变量名，n 是子任务的序号，格式为:
Step{n}: (子任务描述) #E{n} = function_name[kwargs_with_json]

例如：
Step1: 假设 Thomas 工作了 x 小时，将问题转化为代数表达式并使用 Wolfram Alpha 解决。#E1 = WolframAlpha[{"prompt":"Solve x + (2x − 10)

### 查看 Solver 提示语组装结果

In [12]:
print(flow.get_agent("solver").memory[0]['content'])

解决以下任务或问题。
为了解决这个问题，我们制定了逐步计划，并获得了每个计划的执行结果。
请谨慎使用这些执行结果，因为其中可能包含不相关的信息。

Step1: 首先，需要确定鸿蒙公司的具体位置，以便查询其所在城市的天气。
#E1 = get_city[{"location": "鸿蒙公司"}]
#E1 执行结果: 广州

Step2: 使用上一步得到的城市信息，查询该城市的天气情况。
#E2 = get_weather[{"city": "#E1"}]
#E2 执行结果: 广州天气不错，一直是大晴天

Step1: 获取当前所在城市
#E1 = get_city[{"location": "我的位置"}]
#E1 执行结果: 广州

Step2: 查询该城市的天气情况
#E2 = get_weather[{"city": "#E1"}]
#E2 执行结果: 广州天气不错，一直是大晴天

Step3: 根据天气情况决定是否预订球场
#E3 = booking[{"request": "如果 #E2 不包含'雨'，则预订一个足球场"}]
#E3 执行结果: 我已经帮你预订好了，祝你玩得愉快


现在根据上面提供的执行结果解决问题或任务。
直接回答问题，不要废话，不要评论。

任务: 如果不下雨，你帮我订一个球场？

请按如下格式输出你的回答:

```markdown
**最终答案**
(你的回答)
```


### 复杂情况：订球场

In [4]:
flow("如果不下雨，你帮我订一个球场？")

[AGENT] >>> Node 1: planner
Step1: 获取当前位置所在城市 #E1 = get_city[{"location": "我的位置"}]
Step2: 查询该城市的天气情况 #E2 = get_weather[{"city": "#E1"}]
Step3: 根据天气情况决定是否预订球场 #E3 = booking[{"request": "如果 #E2 不包含 '雨'，则预订一个球场"}]
广州
广州天气不错，一直是大晴天
我已经帮你预订好了，祝你玩得愉快
[AGENT] >>> Node 2: observe_func
如果不下雨，你帮我订一个球场？[AGENT] >>> Node 3: solver
```markdown
**最终答案**
我已经帮你预订好了，祝你玩得愉快
```


'我已经帮你预订好了，祝你玩得愉快'

In [13]:
print(flow.planner.memory[0]['content'])

对于以下任务，制定可以逐步解决问题的计划。
对于每个计划，指明使用哪个外部工具以及工具输入来获取证据。

工具可以是以下之一：
{"type": "function", "function": {"name": "get_city", "description": "由任意地址描述查询出所在的城市", "parameters": {"type": "object", "properties": {"location": {"type": "string", "description": ""}}, "required": ["location"]}}}
{"type": "function", "function": {"name": "get_weather", "description": "我可以查询城市的天气情况。city必须是城市名称。", "parameters": {"type": "object", "properties": {"city": {"type": "string", "description": ""}}, "required": ["city"]}}}
{"type": "function", "function": {"name": "booking", "description": "我可以根据你的需求预订球场", "parameters": {"type": "object", "properties": {"request": {"type": "string", "description": ""}}, "required": ["request"]}}}

你可以将具体计划的执行结果存储在一个变量 #E{n} 中，后续工具可以调用该变量。
(Step1, #E1, Step2, #E2, Step3, ...)

其中 #E{n} 用于保存计划执行后的变量名，n 是子任务的序号，格式为:
Step{n}: (子任务描述) #E{n} = function_name[kwargs_with_json]

例如：
Step1: 假设 Thomas 工作了 x 小时，将问题转化为代数表达式并使用 Wolfram Alpha 解决。#E1 = WolframAlpha[{"prompt":"Solve x + (2x − 10)

## Plan And Solve

### 复杂情况：出差、订球场

In [2]:
from illufly.types import BaseAgent, PromptTemplate
from illufly.chat import ChatQwen, PlanAndSolve

def guess_city(prompt: str):
    """由任意地址描述查询出所在的城市"""
    return "广州"

def get_weather(city: str):
    """我可以查询城市的天气情况。city必须是城市名称。"""
    return f'{city}天气不错，一直是大晴天'

def booking(request: str):
    """我可以根据你的需求预订球场"""
    return '我已经帮你预订好了，祝你玩得愉快'

toolkits = [guess_city, get_weather, booking]
planner = ChatQwen(tools=toolkits)
worker = ChatQwen(tools=toolkits)
replanner = ChatQwen(tools=toolkits)

flow = PlanAndSolve(planner, worker, replanner)
flow("我要去见海珠区的客户，看看我是否需要带伞，另外帮我订一个羽毛球场")

[AGENT] >>> Node 1: planner
Step1: 使用 guess_city 函数根据地址描述“海珠区”查询所在城市 #E1
Step2: 使用 get_weather 函数查询 #E1 返回的城市的天气情况 #E2
Step3: 使用 booking 函数根据需求预订羽毛球场 #E3
[AGENT] >>> Node 2: observe_plan_for_worker
[AGENT] >>> Node 3: worker
Step1: 使用 guess_city 函数根据地址描述“海珠区”查询所在城市 #E1 = guess_city[{"prompt": "海珠区"}]
广州
[AGENT] >>> Node 4: observe_worker_for_replanner
[AGENT] >>> Node 5: replanner
**更新计划**

Step1: 使用 get_weather 函数查询广州的天气情况 #E2
Step2: 使用 booking 函数预订一个羽毛球场 #E3
[AGENT] >>> Node 2: observe_plan_for_worker
[AGENT] >>> Node 3: worker
Step1: 使用 get_weather 函数查询广州的天气情况 #E1 = get_weather[{"city": "广州"}]
广州天气不错，一直是大晴天
[AGENT] >>> Node 4: observe_worker_for_replanner
[AGENT] >>> Node 5: replanner
**更新计划**

Step1: 使用 booking 函数预订羽毛球场 #E1

请执行上述计划。
[AGENT] >>> Node 2: observe_plan_for_worker
[AGENT] >>> Node 3: worker
Step1: 使用 booking 函数预订羽毛球场. #E1 = booking[{"request": "预订羽毛球场"}]
我已经帮你预订好了，祝你玩得愉快
[AGENT] >>> Node 4: observe_worker_for_replanner
[AGENT] >>> Node 5: replanner
**最终答案**
根据广州的天气情况，今

'根据广州的天气情况，今天是大晴天，所以你不需要带伞。我已经为你预订了一个羽毛球场，祝你玩得愉快。'

In [3]:
flow.task

'我要去见海珠区的客户，看看我是否需要带伞，另外帮我订一个羽毛球场'

In [4]:
flow.completed_work

['Step1: 使用 guess_city 函数确定海珠区所在的准确城市\nE1 = guess_city[{"prompt": "海珠区"}]\nE1 执行结果: 广州\n',
 'Step1: 使用 get_weather 函数查询广州的天气情况\nE1 = get_weather[{"city": "广州"}]\nE1 执行结果: 广州天气不错，一直是大晴天\n',
 'Step1: 使用 booking 函数预订羽毛球场.\nE1 = booking[{"request": "预订羽毛球场"}]\nE1 执行结果: 我已经帮你预订好了，祝你玩得愉快\n']

### 查看提示语组装结果

In [4]:
print(flow.replanner.memory[0]['content'])

结合可用的工具，为给定目标制定一个简单的逐步执行的计划。
这个计划应包括各个任务，如果正确执行，将得出正确答案。
不要添加任何多余的步骤。
最终步骤的结果应该是最终答案。
确保每个步骤都有所需的所有信息 - 不要跳过步骤。

你可以从 guess_city,get_weather,booking 中选择一个或多个工具使用。这些工具的详细描述为：

{"type": "function", "function": {"name": "guess_city", "description": "由任意地址描述查询出所在的城市，包括公司等一些场景概念，我也可以猜测出具体地址", "parameters": {"type": "object", "properties": {"prompt": {"type": "string", "description": ""}}, "required": ["prompt"]}}}
{"type": "function", "function": {"name": "get_weather", "description": "我可以查询城市的天气情况。city必须是城市名称。", "parameters": {"type": "object", "properties": {"city": {"type": "string", "description": ""}}, "required": ["city"]}}}
{"type": "function", "function": {"name": "booking", "description": "我可以根据你的需求预订球场", "parameters": {"type": "object", "properties": {"request": {"type": "string", "description": ""}}, "required": ["request"]}}}

你的目标是：
请开始


你最新的计划是：


结合之前的计划和最新计划，你目前已经完成了以下步骤：
Step1: 使用 guess_city 函数根据提供的地址描述“海珠区”确定所在城市
E1 = guess_city[{"prompt": "海珠区"}]
E1 执行结果: 广州

Step1: 使用 g